In [ ]:
# default_exp dataset

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import numpy as np
import shutil
import sys
import argparse
import logging
from os.path import join, isdir, isfile, dirname, normpath, basename, splitext
from functools import partial
from datetime import datetime
from logging.handlers import MemoryHandler
from mlcore.core import Type, infer_type
from mlcore.image.pillow_tools import assign_exif_orientation, write_exif_metadata, get_image_size, write_mask
from mlcore.io.core import create_folder, scan_files
from mlcore.annotation.core import RegionShape, convert_region, region_bounding_box
from mlcore.annotation.via_adapter import read_annotations as via_read_annotations
from mlcore.annotation.via_adapter import write_annotations as via_write_annotations
from mlcore.annotation.folder_category_adapter import read_annotations as folder_category_read_annotations
from mlcore.annotation.multi_category_adapter import read_annotations as multi_category_read_annotations
from mlcore.annotation.multi_category_adapter import write_annotations as multi_category_write_annotations
from mlcore.tensorflow.tfrecord_builder import create_tfrecord_file, create_labelmap_file
from mlcore.evaluation.core import box_area, intersection_box, union_box
from mlcore.category_tools import read_categories

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

CATEGORY_LABEL_KEY = 'category'
DEFAULT_CATEGORIES_FILE = 'categories.txt'
DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE = 'annotations.csv'
DEFAULT_SEGMENTATION_ANNOTATIONS_FILE = 'via_region_data.json'
DEFAULT_SPLIT = 0.2
DATA_SET_FOLDER = 'datasets'
SEMANTIC_MASK_FOLDER = 'semantic_masks'
TRAIN_VAL_FOLDER = 'trainval'
TRAIN_FOLDER = 'train'
VAL_FOLDER = 'val'
TEST_FOLDER = 'test'
NOT_CATEGORIZED = '[NOT_CATEGORIZED]'

In [ ]:
# export

logger = logging.getLogger(__name__)

# Data-Set

> Data-Set Notes.

Creating a data-set for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The data-set is created based on an image-set.

## Image-Set

Image-sets are collected images to build a data-set from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[image_set_type]*/*[image_set_name]*

Inside the `[image_set_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the image-set contains

## Data-Set Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[data_set_type]*/*[data_set_name]*
where `[data_set_type]` is the same as the corresponding `[image_set_type]` and `[data_set_name]` is the same as the corresponding `[image_set_name]`.

Inside the `[data_set_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the data-set contains

In [ ]:
# export


class DataSet:
    def __init__(self, name, base_path, image_set_path, categories_path, data_set_type, create_tfrecord=False):
        self.name = name
        self.base_path = base_path
        self.image_set_path = image_set_path
        self.train_val_folder = join(image_set_path, TRAIN_VAL_FOLDER)
        self.test_source_folder = join(image_set_path, TEST_FOLDER)
        self.categories_path = categories_path
        self.categories = read_categories(categories_path, data_set_type)
        self.type = data_set_type
        self.create_tfrecord = create_tfrecord
        self.folder = None
        self.train_folder = None
        self.val_folder = None
        self.test_target_folder = None
        self.logger = self.get_logger()
        self.annotations = None

    def create_folders(self):
        """
        Creates the data-set folder structure, if not exist
        """

        # create data-set folder and remove previous data if exist
        self.folder = create_folder(join(self.base_path, str(self.type), self.name), clear=True)
        self.logger.info("Created folder {}".format(self.folder))
        self.train_folder = create_folder(join(self.folder, TRAIN_FOLDER))
        self.logger.info("Created folder {}".format(self.train_folder))
        self.val_folder = create_folder(join(self.folder, VAL_FOLDER))
        self.logger.info("Created folder {}".format(self.val_folder))
        if isdir(self.test_source_folder):
            self.test_target_folder = create_folder(join(self.folder, TEST_FOLDER))
            self.logger.info("Created folder {}".format(self.test_target_folder))

    def convert_annotations(self):
        """
        Converts annotations
        """
        pass

    def validate(self):
        """
        Validates, that each file has at least one annotation.
        """

        # validate only the trainval images, the test images have no annotations to validate
        # convert the annotations before doing validation
        self.convert_annotations()

        self.logger.info('Start validate image set at {}'.format(self.train_val_folder))

        files = scan_files(self.train_val_folder)

        self.logger.info('Found {} files at {}'.format(len(files), self.train_val_folder))

        delete_annotations = {}
        used_categories = set([])

        for annotation_id, annotation in self.annotations.items():
            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                len_labels = len(region.labels)
                region_valid = len_labels and len(set(region.labels) & set(self.categories)) == len_labels
                if not region_valid:
                    message = '{} : Region {} with category {} is not in category list, skip region.'
                    self.logger.info(message.format(annotation.file_name, index, ','.join(region.labels)))
                    delete_regions[index] = True
                else:
                    # update the used regions
                    used_categories.update(region.labels)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

            # validate for empty region
            if not annotation.regions:
                self.logger.info('{} : Has empty regions, skip annotation.'.format(annotation.file_path))
                delete_annotations[annotation_id] = True
            # validate for file exist
            elif annotation.file_path not in files:
                self.logger.info('{} : File of annotations do not exist, skip annotations.'.format(annotation.file_path))
                delete_annotations[annotation_id] = True
            else:
                files.pop(files.index(annotation.file_path))

        for index, file in enumerate(files):
            self.logger.info('[{}] -> {} : File has no annotations, skip file.'.format(index, file))

        # list unised categories
        empty_categories = frozenset(self.categories) - used_categories
        if empty_categories:
            self.logger.info('The following categories have no images: {}'.format(" , ".join(empty_categories)))

        # delete annotations after iteration is finished
        for index in delete_annotations.keys():
            del self.annotations[index]

        self.logger.info('Finished validate image set at {}'.format(self.train_val_folder))

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        self.logger.info('Start copy files from {} to {}'.format(self.image_set_path, self.folder))

        # copy the categories files
        self.logger.info('Copy file {} to {}'.format(self.categories_path, self.folder))
        shutil.copy2(self.categories_path, join(self.folder, DEFAULT_CATEGORIES_FILE))

        # if create tfrecord, create a labelmap.pbtxt file containing the categories
        if self.create_tfrecord:
            labelmap_file_name = 'label_map.pbtxt'
            labelmap_output_file = join(self.folder, labelmap_file_name)
            self.logger.info('Generate file {} to {}'.format(labelmap_file_name, self.folder))
            create_labelmap_file(labelmap_output_file, list(self.categories)[1:], 1)

        annotations_train = {}
        annotations_val = {}

        num_files = len(train_file_keys)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.train_folder))

        for key in train_file_keys:
            # copy image
            annotation = self.annotations[key]
            copy_image_and_assign_orientation(self.train_val_folder, annotation.file_name, self.train_folder)

            # add annotation
            annotations_train[key] = annotation
        self.logger.info('Finished copy {} files to {}'.format(num_files, self.train_folder))

        num_files = len(val_file_keys)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.val_folder))

        for key in val_file_keys:
            # copy image
            annotation = self.annotations[key]
            copy_image_and_assign_orientation(self.train_val_folder, annotation.file_name, self.val_folder)

            # add annotation
            annotations_val[key] = annotation

        self.logger.info('Finished copy {} files to {}'.format(num_files, self.val_folder))

        # copy test_files, if exist
        if test_file_names:

            num_files = len(test_file_names)
            self.logger.info('Start copy {} files to {}'.format(num_files, self.test_target_folder))

            for file_name in test_file_names:
                copy_image_and_assign_orientation(self.test_source_folder, file_name, self.test_target_folder)

            self.logger.info('Finished copy {} files to {}'.format(num_files, self.test_target_folder))

        self.logger.info('Finished copy files from {} to {}'.format(self.image_set_path, self.folder))

        return annotations_train, annotations_val

    def build(self, split=DEFAULT_SPLIT, seed=None, sample=None):
        """
        Build the data-set. This is the main logic.
        This method validates the images against the annotations,
        split the image-set into train and val on given split percentage,
        creates the data-set folders and copies the image.
        If a sample percentage is given, a sub-set is created as sample.
        `split`: The percentage of images which will be copied into the validation set
        `seed`: A random seed to reproduce splits
        `sample`: The percentage of images from train, val and test which will also from a sample set
        """

        self.logger.info('Validation set contains {}% of the images.'.format(int(split * 100)))

        # validate the image set
        self.validate()

        # sort filenames by assigned tags
        labels_to_keys = {}

        for annotation_id, annotation in self.annotations.items():
            labels = ' '.join(annotation.labels())

            if labels not in labels_to_keys:
                labels_to_keys[labels] = []
            labels_to_keys[labels].append(annotation_id)

        # split category files into train & val and create the sample split, if set
        train_file_keys = []
        val_file_keys = []
        sample_train_file_keys = []
        sample_val_file_keys = []

        for annotation_ids in labels_to_keys.values():
            train, val = split_train_val_data(annotation_ids, split, seed)
            train_file_keys.extend(train)
            val_file_keys.extend(val)

            # if a sample data set should be created, create the splits
            if sample:
                _, sample_train = split_train_val_data(train, sample, seed)
                _, sample_val = split_train_val_data(val, sample, seed)
                sample_train_file_keys.extend(sample_train)
                sample_val_file_keys.extend(sample_val)

        # scan the test images if exist
        test_file_names = list(map(lambda f: basename(f), scan_files(self.test_source_folder))) if self.test_target_folder else None
        _, sample_test_file_names = split_train_val_data(test_file_names, sample, seed) if test_file_names and sample else (None, None)

        # copy the files
        self.copy(train_file_keys, val_file_keys, test_file_names)

        if sample:
            sample_name = "{}_sample".format(self.name)

            self.logger.info('Start build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                             int(sample * 100),
                                                                                             self.base_path))

            # create the sample data-set
            sample_data_set = self.__class__(sample_name, self.base_path, self.image_set_path, self.categories_path,
                                             self.type, create_tfrecord=self.create_tfrecord)
            # assign the converted annotations
            sample_data_set.annotations = self.annotations

            # create the data set folders
            sample_data_set.create_folders()
            # copy the sample images
            sample_data_set.copy(sample_train_file_keys, sample_val_file_keys, sample_test_file_names)

            self.logger.info('Finished build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                                int(sample * 100),
                                                                                                self.base_path))

    @classmethod
    def get_logger(cls):
        """
        Configures default logging for the system.
        """
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
        return logger

## Create a classification data-set

Classification data-set can be created from a classification image-set. 
All images are validated, if they belong to one of the given categories. If categories with no images are found or images belong to a category not listed in `categories.txt`, the data-set can not be created.

In [ ]:
# export


class ClassificationDataSet(DataSet):
    """
    Classification data-set.
    `name`: The name of the data-set.
    `base_path`: The data-set base-path.
    `image_set_path`: The image-set source path.
    `categories_path`: The path to the categories.txt file.
    `data_set_type`: The type of the data-set.
    `annotations_path`: The path to the annotations-file.
    """

    def __init__(self, name, base_path, image_set_path, categories_path, data_set_type, annotations_path=None,
                 create_tfrecord=False):
        super().__init__(name, base_path, image_set_path, categories_path, data_set_type, create_tfrecord)
        self.annotations_path = annotations_path
        # if no annotation file exist, generate annotations based on the folder names
        if not annotations_path:
            self.annotations = folder_category_read_annotations(self.train_val_folder)
        else:
            self.annotations = multi_category_read_annotations(annotations_path, self.train_val_folder)

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)

        # write the split train annotations
        if annotations_train:
            annotations_target_path = join(self.train_folder, DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            multi_category_write_annotations(annotations_target_path, annotations_train)

        # write the split val annotations
        if annotations_val:
            annotations_target_path = join(self.val_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            multi_category_write_annotations(annotations_target_path, annotations_val)

        return annotations_train, annotations_val

## Create a object detection data-set

Object detection data-set can be created from a segmentation or object-detection image-set.
All images are validated against the annotations, if they contain at least one annotation and that the annotation category belongs to one of the given categories. The annotations have to be in [VIA v1](http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.5.html) json format. Polygon annotations are converted into rectangle annotations for unique bounding-box generation.

In [ ]:
# export


class ObjectDetectionDataSet(DataSet):
    """
    Object detection data-set.
    `name`: The name of the data-set.
    `base_path`: The data-set base-path.
    `image_set_path`: The image-set source path.
    `categories_path`: The path to the categories.txt file.
    `data_set_type`: The type of the data-set.
    `annotations_path`: The path to the annotations-file.
    `create_tfrecord`: Also create .tfrecord files.
    `join_overlapping_regions`: Whether overlapping regions of same category should be joined.
    `annotation_area_threshold`: Keep only annotations with minimum size (width or height) related to image size
    """

    def __init__(self, name, base_path, image_set_path, categories_path, data_set_type, annotations_path=None,
                 create_tfrecord=False, join_overlapping_regions=False, annotation_area_threshold=None):
        super().__init__(name, base_path, image_set_path, categories_path, data_set_type, create_tfrecord)
        self.annotations_path = annotations_path
        self.annotations = via_read_annotations(annotations_path, self.train_val_folder, CATEGORY_LABEL_KEY) if annotations_path else {}
        self.join_overlapping_regions = join_overlapping_regions
        self.annotation_area_threshold = annotation_area_threshold

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)

        # write the split train annotations
        if annotations_train:
            annotations_target_path = join(self.train_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            via_write_annotations(annotations_target_path, annotations_train, CATEGORY_LABEL_KEY)
            # if creating a .tfrecord
            if self.create_tfrecord:
                tfrecord_file_name = '{}.record'.format(normpath(basename(self.train_folder)))
                tfrecord_output_file = join(self.folder, tfrecord_file_name)
                self.logger.info('Generate file {} to {}'.format(tfrecord_file_name, self.folder))
                create_tfrecord_file(tfrecord_output_file, self.categories, annotations_train)

        # write the split val annotations
        if annotations_val:
            annotations_target_path = join(self.val_folder, DEFAULT_SEGMENTATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            via_write_annotations(annotations_target_path, annotations_val, CATEGORY_LABEL_KEY)
            # if creating a .tfrecord
            if self.create_tfrecord:
                tfrecord_file_name = '{}.record'.format(normpath(basename(self.val_folder)))
                tfrecord_output_file = join(self.folder, tfrecord_file_name)
                self.logger.info('Generate file {} to {}'.format(tfrecord_file_name, self.folder))
                create_tfrecord_file(tfrecord_output_file, self.categories, annotations_val)

        return annotations_train, annotations_val

    def convert_annotations(self):
        """
        Converts segmentation regions from polygon to rectangle, if exist
        """

        # only the trainval images have annotation, not the test images
        area_threshold = self.annotation_area_threshold

        steps = [
            {
                'name': 'position',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    't': 'Trim',  # transform the annotation
                    'T': 'Trim All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: p_min < 0 or p_max >= size,
                'message': '{} -> {} : {}Exceeds image {}. \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}',
                'transform': lambda p, size=0: max(min(p, size - 1), 0),
            },
            {
                'name': 'size',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, _: p_max - p_min <= 1,
                'message': '{} -> {} : {}Shape {} is <= 1 pixel. \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}',
                'transform': lambda p, size=0: p,
            },
            {
                'name': 'area',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: area_threshold and (p_max - p_min) / size <= area_threshold,
                'message': '{} <= {} percent. {}'.format('{} -> {} : {}Shape {} is', (area_threshold or 0) * 100, ' \n Box \n x: {} \n y: {} \n x_max: {} \n y_max: {}'),
                'transform': lambda p, size=0: p,
            }
        ]

        self.logger.info('Start convert image annotations from {}'.format(self.annotations_path))

        for annotation in self.annotations.values():
            # skip file, if regions are empty or file do not exist
            if not (annotation.regions and isfile(annotation.file_path)):
                continue

            # convert from polygon to rect if needed
            for region in annotation.regions:
                convert_region(region, RegionShape.RECTANGLE)

            # try to join regions
            if self.join_overlapping_regions:
                self.join_regions(annotation.regions)

            image, _, __ = assign_exif_orientation(annotation.file_path)
            img_width, img_height = image.size
            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                # validate the shape size
                x_min, x_max = region.points_x[:2]
                y_min, y_max = region.points_y[:2]
                for step in steps:
                    width_condition = step['condition'](x_min, x_max, img_width)
                    height_condition = step['condition'](y_min, y_max, img_height)
                    if width_condition or height_condition:
                        size_message = ['width'] if width_condition else []
                        size_message.extend(['height'] if height_condition else [])
                        message = step['message'].format(annotation.file_name, index, ' ',
                                                         ' and '.join(size_message),
                                                         x_min, y_min, x_max, y_max)

                        step['choice'] = input_feedback(message, step['choice'], step['choices'])

                        choice_op = step['choice'].lower()
                        # if skip the shapes
                        if choice_op == 's':
                            delete_regions[index] = True
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             x_min, y_min, x_max, y_max)
                            self.logger.info(message)

                            break
                        else:
                            region.points_x = list(map(partial(step['transform'], size=img_width), [x_min, x_max]))
                            region.points_y = list(map(partial(step['transform'], size=img_height), [y_min, y_max]))
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             x_min, y_min, x_max, y_max)
                            self.logger.info(message)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

        self.logger.info('Finished convert image annotations from {}'.format(self.annotations_path))

    def join_regions(self, regions):
        """
        Join regions which overlaps.
        `regions`: the region to parse
        """
        len_before = len(regions)
        index_left = 0
        while index_left < len(regions):
            regions_joined = []
            region_left = regions[index_left]
            for index_right in range(len(regions)):
                if index_left == index_right:
                    continue
                region_right = regions[index_right]
                same_label_length = len(region_left.labels) == len(region_right.labels)
                same_labels = same_label_length and len(region_left.labels) == len(set(region_left.labels) & set(region_right.labels))
                if same_labels:
                    bbox_left = (region_left.points_x, region_left.points_y)
                    bbox_right = (region_right.points_x, region_right.points_y)
                    inter_area = box_area(intersection_box(bbox_left, bbox_right))
                    if inter_area > 0:
                        points_x, points_y = union_box(bbox_left, bbox_right)
                        region_left.points_x = points_x
                        region_left.points_y = points_y
                        regions_joined.append(index_right)
            for index in regions_joined[::-1]:
                del regions[index]
            if not regions_joined:
                index_left += 1
        self.logger.info('Joined overlapping regions from {} -> {}.'.format(len_before, len(regions)))

## Create a segmentation data-set

Segmentation data-set can be created from a segmentation image-set.
All images are validated against the annotations, if they contain at least one annotation and that the annotation category belongs to one of the given categories. The annotations have to be in [VIA v1](http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.5.html) json format. Rectangle annotations are converted into polygon annotations for unique segment generation.

In [ ]:
# export


class SegmentationDataSet(ObjectDetectionDataSet):
    def __init__(self, name, base_path, image_set_path, categories_path, data_set_type, annotations_path=None):
        super().__init__(name, base_path, image_set_path, categories_path, data_set_type, annotations_path)
        self.semantic_mask_folder = None

    def create_folders(self):
        """
        Creates the data-set folder structure, if not exist
        """
        super().create_folders()

        # create semantic mask file folder and remove previous data if exist
        self.semantic_mask_folder = create_folder(join(self.folder, SEMANTIC_MASK_FOLDER), clear=True)
        self.logger.info("Created folder {}".format(self.semantic_mask_folder))

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images
        and generate the semantic masks.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)
        # save semantic masks
        self._save_semantic_masks(train_file_keys + val_file_keys)
        return annotations_train, annotations_val

    def convert_annotations(self):
        """
        Converts segmentation regions from rectangle to polygon, if exist
        """

        # only the trainval images have annotation, not the test images
        steps = [
            {
                'name': 'position',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    't': 'Trim',  # transform the annotation
                    'T': 'Trim All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: p_min < 0 or p_max >= size,
                'message': '{} -> {} : {}Exceeds image {}. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: max(min(p, size - 1), 0),
            },
            {
                'name': 'size',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, _: p_max - p_min <= 1,
                'message': '{} -> {} : {}Shape {} is <= 1 pixel. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: p,
            }
        ]

        self.logger.info('Start convert image annotations from {}'.format(self.annotations_path))

        for annotation in self.annotations.values():
            # skip file, if regions are empty or file do not exist
            if not (annotation.regions and isfile(annotation.file_path)):
                continue

            image, _, __ = assign_exif_orientation(annotation.file_path)
            image_width, image_height = image.size

            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                # convert from rect to polygon if needed
                convert_region(region, RegionShape.POLYGON)

                for step in steps:
                    # validate the shape size
                    (x_min, x_max), (y_min, y_max) = region_bounding_box(region)

                    width_condition = step['condition'](x_min, x_max, image_width)
                    height_condition = step['condition'](y_min, y_max, image_height)
                    if width_condition or height_condition:
                        size_message = ['width'] if width_condition else []
                        size_message.extend(['height'] if height_condition else [])
                        message = step['message'].format(annotation.file_name, index, ' ', ' and '.join(size_message),
                                                         region.points_x, region.points_y)

                        step['choice'] = input_feedback(message, step['choice'], step['choices'])

                        choice_op = step['choice'].lower()
                        # if skip the shapes
                        if choice_op == 's':
                            delete_regions[index] = True
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             region.points_x, region.points_y)
                            self.logger.info(message)

                            break
                        else:
                            annotation.points_x = list(map(partial(step['transform'], size=image_width),
                                                           annotation.points_x))
                            annotation.points_y = list(map(partial(step['transform'], size=image_height),
                                                           annotation.points_y))

                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             region.points_x, region.points_y)
                            self.logger.info(message)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

        print('Finished convert image annotations from {}'.format(self.annotations_path))

    def _save_semantic_masks(self, keys):
        """
        Create semantic segmentation mask png files out of the annotations.
        The mask file name is the same as the image file name but is stored in png format.
        `keys`: The annotation keys to create the segmentation masks for
        """
        from skimage import draw

        num_masks = len(keys)
        self.logger.info('Start create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

        # only the trainval images have annotation, not the test images
        for index, key in enumerate(keys):
            annotation = self.annotations[key]

            if not annotation.annotations:
                continue

            image, image_width, image_height = get_image_size(annotation.file_path)

            # Convert polygons to a bitmap mask of shape
            # [height, width]
            mask = np.zeros((image_height, image_width), dtype=np.uint8)

            # sort the regions by category priority for handling pixels which are assigned to more than one category
            # the category with higher index overpaint the category with lower index
            for region in sorted(annotation.regions(), key=lambda r: self.categories.index(r.labels[0])):
                class_id = self.categories.index(region.labels[0]) + 1

                # Get indexes of pixels inside the polygon and set them to 1
                rr, cc = draw.polygon(region.points_y, region.points_x)
                mask[rr, cc] = class_id

            # save the semantic mask
            mask_path = join(self.semantic_mask_folder, splitext(annotation.file_name)[0] + '.png')
            write_mask(mask, mask_path)

            self.logger.info('{} / {} - Created segmentation mask {}'.format(index + 1, num_masks, mask_path))

        self.logger.info('Finish create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

## Helper Methods

In [ ]:
# export


def split_train_val_data(data, val_size=0.2, seed=None):
    """
    Splits the images in train and validation set
    `data`: the data to split
    `val_size`: the size of the validation set in percentage
    `seed`: A random seed to reproduce splits.
    return: the splittet train, validation images
    """

    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, random_state=seed, test_size=val_size) if len(data) > 1 else (data, [])
    return train, test

In [ ]:
# export


def copy_image_and_assign_orientation(source_folder, file_name, target_folder):
    """
    Copy an image file from source to target and assign the EXIF metadata orientation.
    `source_folder`: the source folder of the image file
    `file_name`: the image file name to copy
    `target_folder`: the target folder to copy the image file to
    return: `True` if modified EXIF metadata could be saved, else `False`
    """

    shutil.copy2(join(source_folder, file_name), target_folder)

    # rotate image by EXIF orientation metadata and remove them
    target_file = join(target_folder, file_name)
    image, exif_data, rotated = assign_exif_orientation(target_file)
    if rotated:
        write_exif_metadata(image, exif_data, target_file)

In [ ]:
# export


def input_feedback(msg, choice, choices):
    """
    User input request wrapper.
    `msg`: the message to dosplay
    `choice`: if previous choice exist
    `choices`: the possible choices
    :return: the choice input
    """

    # if decision is already made for all contents, skip feedback
    if not (choice and choice.isupper()):
        prompt = '{} \n choices: {} '.format(msg, ', '.join(['{} ({})'.format(k, v) for k, v in choices.items()]))
        while True:
            choice = input(prompt)
            if choice in choices:
                break
    return choice

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.
    """
    log_memory_handler = MemoryHandler(1, flushLevel=logging_level)
    log_memory_handler.setLevel(logging_level)

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging_level)

    logger.addHandler(log_memory_handler)
    logger.addHandler(stdout_handler)

    logger.setLevel(logging_level)

    return log_memory_handler

## Build a data-set

To build a data-set from an image-set. Handles currently classification and segmentation image-sets taken from the image-set-type, which is the parent folder, the image-set folder is located in. 

In [ ]:
# export


def build_data_set(category_file_path, output, annotation_file_path=None, split=DEFAULT_SPLIT, seed=None, sample=0,
                   data_set_type=None, create_tfrecord=False, join_overlapping_regions=False,
                   annotation_area_threshold=None, data_set_name=None):
    """
    Build the data-set for training, Validation and test
    `category_file_path`: the filename of the categories file
    `output`: the dataset base folder to build the dataset in
    `annotation_file_path`: the file path to the annotation file
    `split`: the size of the validation set as percentage
    `seed`: random seed to reproduce splits
    `sample`: the size of the sample set as percentage
    `data_set_type`: the type of the data-set, if not set infer from the category file path
    `create_tfrecord`: Also create .tfrecord files.
    `join_overlapping_regions`: Whether overlapping regions of same category should be joined.
    `annotation_area_threshold`: Keep only annotations with minimum size (width or height) related to image size
    `data_set_name`: the name of the data-set, if not set infer from the category file path
    """
    log_memory_handler = configure_logging()

    # try to infer the data-set type if not explicitly set
    if data_set_type is None:
        try:
            data_set_type = infer_type(category_file_path)
        except ValueError as e:
            logger.error(e)
            return

    path = dirname(category_file_path)

    # try to infer the data-set name if not explicitly set
    if data_set_name is None:
        data_set_name = basename(path)

    logger.info('Build parameters:')
    logger.info(' '.join(sys.argv[1:]))
    logger.info('Build configuration:')
    logger.info('category_file_path: {}'.format(category_file_path))
    logger.info('annotation_file_path: {}'.format(annotation_file_path))
    logger.info('split: {}'.format(split))
    logger.info('seed: {}'.format(seed))
    logger.info('sample: {}'.format(sample))
    logger.info('type: {}'.format(data_set_type))
    logger.info('output: {}'.format(output))
    logger.info('join_overlapping_regions: {}'.format(join_overlapping_regions))
    logger.info('annotation_area_threshold: {}'.format(annotation_area_threshold))
    logger.info('name: {}'.format(data_set_name))

    data_set = None
    logger.info('Start build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

    if data_set_type == Type.IMAGE_CLASSIFICATION:
        data_set = ClassificationDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                         annotation_file_path)
    elif data_set_type == Type.IMAGE_SEGMENTATION:
        data_set = SegmentationDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                       annotation_file_path)
    elif data_set_type == Type.IMAGE_OBJECT_DETECTION:
        data_set = ObjectDetectionDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                          annotation_file_path, create_tfrecord, join_overlapping_regions,
                                          annotation_area_threshold)

    if data_set:
        # create the data set folders
        logger.info("Start create the data-set folders at {}".format(data_set.base_path))
        data_set.create_folders()
        logger.info("Finished create the data-set folders at {}".format(data_set.base_path))

        # create the build log file
        log_file_name = datetime.now().strftime("build_%Y.%m.%d-%H.%M.%S.log")
        file_handler = logging.FileHandler(join(data_set.folder, log_file_name), encoding="utf-8")
        log_memory_handler.setTarget(file_handler)

        # build the data set
        data_set.build(split, seed, sample)

    logger.info('Finished build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.dataset [parameters]`

The following parameters are supported:
- `[categories]`: The path to the categories file. (e.g.: *categories.txt*)
- `--annotation`: The path to the image-set annotation file, the data-set is build from. (e.g.: *imagesets/classification/car_damage/annotations.csv* for classification, *imagesets/segmentation/car_damage/via_region_data.json* for segmentation)
- `--split`: The percentage of the data which belongs to validation set, default to *0.2* (=20%)
- `--seed`: A random seed to reproduce splits, default to None
- `--category-label-key`: The key, the category name can be found in the annotation file, default to *category*.
- `--sample`: The percentage of the data which will be copied as a sample set with in a separate folder with "_sample" suffix. If not set, no sample data-set will be created.
- `--type`: The type of the data-set, if not explicitly set try to infer from categories file path.
- `--tfrecord`: Also create .tfrecord files.
- `--join-overlapping-regions`: Whether overlapping regions of same category should be joined.
- `--annotation-area-thresh`: Keep only annotations with minimum size (width or height) related to image size.
- `--output`: The path of the dataset folder, default to *../datasets*.
- `--name`: The name of the data-set, if not explicitly set try to infer from categories file path.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("categories",
                        help="The path to the image-set categories file.")
    parser.add_argument("--annotation",
                        help="The path to the image-set annotation file, the data-set is build from.",
                        default=None)
    parser.add_argument("--split",
                        help="Percentage of the data which belongs to validation set.",
                        type=float,
                        default=0.2)
    parser.add_argument("--seed",
                        help="A random seed to reproduce splits.",
                        type=int,
                        default=None)
    parser.add_argument("--category-label-key",
                        help="The key of the category name.",
                        default=CATEGORY_LABEL_KEY)
    parser.add_argument("--sample",
                        help="Percentage of the data which will be copied as a sample set.",
                        type=float,
                        default=0)
    parser.add_argument("--type",
                        help="The type of the data-set, if not explicitly set try to infer from categories file path.",
                        choices=list(Type),
                        type=Type,
                        default=None)
    parser.add_argument("--tfrecord",
                        help="Also create .tfrecord files.",
                        action="store_true")
    parser.add_argument("--join-overlapping-regions",
                        help="Whether overlapping regions of same category should be joined.",
                        action="store_true")
    parser.add_argument("--annotation-area-thresh",
                        help="Keep only annotations with minimum size (width or height) related to image size.",
                        type=float,
                        default=None)
    parser.add_argument("--output",
                        help="The path of the data-set folder.",
                        default=DATA_SET_FOLDER)
    parser.add_argument("--name",
                        help="The name of the data-set, if not explicitly set try to infer from categories file path.",
                        default=None)
    args = parser.parse_args()

    CATEGORY_LABEL_KEY = args.category_label_key

    build_data_set(args.categories, args.output, args.annotation, args.split, args.seed, args.sample, args.type,
                   args.tfrecord, args.join_overlapping_regions, args.annotation_area_thresh, args.name)
